# SSD Tensorflow 2 Object Detection

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
DATA_DIR = '/content/gdrive/My Drive/cv/datasets/test/videos'

In [3]:
# Clone Tensorflow Models Repository
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 87839, done.
remote: Counting objects: 100% (1547/1547), done.
remote: Compressing objects: 100% (664/664), done.
remote: Total 87839 (delta 970), reused 1407 (delta 868), pack-reused 86292
Receiving objects: 100% (87839/87839), 599.40 MiB | 6.90 MiB/s, done.
Resolving deltas: 100% (62881/62881), done.


In [4]:
# install Protobuf (for serialization)
!cd models/research/ && protoc object_detection/protos/*.proto --python_out=.

In [5]:
# Install the object detection API
!cd models/research && cp object_detection/packages/tf2/setup.py . && \
    python -m pip install .

Processing /content/models/research
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 85.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 62.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.6/116.6 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 88.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... don

In [6]:
# Install libs
import os
import pathlib
import imageio
import tensorflow as tf

import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

%matplotlib inline

In [7]:
# make sure we're using GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
# Enable GPU dynamic memory allocation
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

## Download The videos

In [8]:
!cp -r '{DATA_DIR}' .

In [9]:
!ls videos

crossing.mp4  dog.mp4  traffic.mp4


In [10]:
video_PATHS = ['videos/crossing', 'videos/dog', 'videos/traffic']

## Download the model
For a complete list op models to use [TF2.0 Zoo Models](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md)

In [12]:
base_url = "http://download.tensorflow.org/models/object_detection/tf2/"
MODEL_DATE = '20200711'
MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8'
# MODEL_NAME = 'ssd_resnet101_v1_fpn_1024x1024_coco17_tpu-8'
MODEL_EXT  = '.tar.gz'
url = f"{base_url}{MODEL_DATE}/{MODEL_NAME}{MODEL_EXT}"

PATH_TO_MODEL_DIR = tf.keras.utils.get_file(
    fname=MODEL_NAME,
    origin=url,
    untar=True)

20518283/20518283 [==============================] - 0s 0us/step


In [13]:
PATH_TO_MODEL_DIR

'/root/.keras/datasets/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8'

## Download the labels
Label files can be found here: https://github.com/tensorflow/models/tree/master/research/object_detection/data

(Object Detection Zoo contains only models trained on COCO.)

In [14]:
url = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/mscoco_label_map.pbtxt'

PATH_TO_LABELS = tf.keras.utils.get_file(
    fname='mscoco_label_map.pbtxt',
    origin=url,
    untar=False)

5056/5056 [==============================] - 0s 0us/step


In [15]:
PATH_TO_LABELS

'/root/.keras/datasets/mscoco_label_map.pbtxt'

In [16]:
# CHeck labels format
!head {PATH_TO_LABELS}

item {
  name: "/m/01g317"
  id: 1
  display_name: "person"
}
item {
  name: "/m/0199g"
  id: 2
  display_name: "bicycle"
}


## Load the model

In [17]:
PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "/saved_model"

print('Loading model...', end='')
start_time = time.time()

# Load saved model and build the detection function
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

Loading model...Done! Took 15.128544330596924 seconds


## Load the labels

In [18]:
category_index = label_map_util.create_category_index_from_labelmap(
    PATH_TO_LABELS,
    use_display_name=True)

## Putting it all together
Fırst we need to define a function that annotates and returnes a single image

In [19]:
# Annotate a single image

def detect_objects_in_image(image_np):
    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image_np)

    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]

    # input_tensor = np.expand_dims(image_np, 0)
    detections = detect_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=200,
          min_score_thresh=.30,
          agnostic_mode=False)
    return image_np_with_detections

Then we need to call the `detect_objects_in_image` function in a loop and annotate all the frames,images, in the video. Here we assume that the input video is always an mp4 file. To save the result, we're using the `imageio` library to read frames from the input video and turn the results into a new video.

In [20]:
def detect_objects_in_video(input_video):
    print(f'Running inference for {input_video}.mp4... ', end='')

    video_reader = imageio.get_reader(f'{input_video}.mp4')
    video_writer = imageio.get_writer(f'{input_video}_annotated.mp4', fps=10)

    # loop through and process each frame
    t0 = time.time()
    n_frames = 0
    for frame in video_reader:
        n_frames += 1
        new_frame = detect_objects_in_image(frame)

        # instead of plotting image, we write the frame to video
        video_writer.append_data(new_frame)

    fps = n_frames / (time.time() - t0)
    print("Frames processed: %s, Speed: %s fps" % (n_frames, fps))

    # clean up
    video_writer.close()

In [21]:
# for i,video in enumerate(video_PATHS):
#     print('Running inference for {}... '.format(video))
#     detect_objects_in_video(video_PATHS[i])

In [22]:
detect_objects_in_video(video_PATHS[2])

Running inference for videos/traffic.mp4... Frames processed: 551, Speed: 1.9890478435305574 fps
